### Loading the necessary libraries

In [1]:
from __future__ import annotations

import os
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY is not set. Create a .env file with OPENAI_API_KEY=...")

env_model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

SUPPORTED_ASSISTANT_MODELS = {
    "gpt-4o",
    "gpt-4o-mini",
    "gpt-4-turbo",
    "gpt-4-turbo-preview",
    "gpt-4",
    "gpt-3.5-turbo",
}

model_chat = env_model if env_model in SUPPORTED_ASSISTANT_MODELS else "gpt-4o-mini"
model_embedding = os.getenv("OPENAI_MODEL_EMBEDDING")

vector_store_id_env = os.getenv("VECTOR_STORE_ID")


### Creating an OpenAI client

In [2]:
client = OpenAI(api_key=api_key)

print(f"Client ready. Model (chat): {model_chat}")
if model_embedding:
    print(f"Embedding model: {model_embedding}")
if vector_store_id_env:
    print(f"VECTOR_STORE_ID found in env: {vector_store_id_env}")


Client ready. Model (chat): gpt-4o-mini
Embedding model: text-embedding-3-small


### Vector store helpers

In [3]:
def list_vector_stores(limit: int = 100):
    stores = client.vector_stores.list(limit=limit)
    print(f"Total vector stores returned: {len(stores.data)}")
    for vs in stores.data:
        print(f"{vs.id} | {vs.name} | created_at={vs.created_at}")
    return stores


def create_or_get_vector_store(name: str, description: str | None = None):
    if not name.strip():
        raise ValueError("Vector store name must be non-empty")

    stores = client.vector_stores.list(limit=100)
    for vs in stores.data:
        if vs.name == name:
            print("Vector store already exists:")
            print(f"  Name: {vs.name}")
            print(f"  ID:   {vs.id}")
            return vs

    vs = client.vector_stores.create(name=name, description=description)
    print("Vector store created:")
    print(f"  Name: {vs.name}")
    print(f"  ID:   {vs.id}")
    return vs


### Reading the number of vector stores

In [15]:
_ = list_vector_stores()


Total vector stores returned: 1
vs_696a7e49876c81919f5674dfc9a21ad8 | PU Repo | created_at=1768586825


### Searching the vector store

In [ ]:
QUERY = "what is the document about"  # set a query before running

if not QUERY.strip():
    print("QUERY is empty. Set QUERY to a non-empty string to run a search.")
else:
    results = client.vector_stores.search(vector_store_id=vector_store_id, query=QUERY)
    print(f"Results: {len(results.data)}")

    for i, result in enumerate(results.data, start=1):
        print(f"Result {i} | score={result.score}")
        if result.content and getattr(result.content[0], "text", None):
            print(result.content[0].text)
        print("-" * 50)